# Self-Training with Semi-Supervised Learning on Wine Quality Dataset

This notebook demonstrates Self-Training, a semi-supervised learning technique. We simulate a realistic scenario using the Wine Quality dataset by masking most of the labels and training a classifier using both labeled and unlabeled data.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay

sns.set(style="whitegrid")

In [ ]:
# Load dataset
df = pd.read_csv("winequality-red.csv", sep=';')
df.head()

## Exploratory Data Analysis (EDA)

In [ ]:
print("Shape:", df.shape)
print("\nMissing Values:\n", df.isnull().sum())
print("\nQuality Distribution:\n", df['quality'].value_counts().sort_index())

In [ ]:
# Class distribution
plt.figure(figsize=(8, 4))
sns.countplot(x='quality', data=df, palette='Set2')
plt.title('Wine Quality Distribution')
plt.xlabel('Quality Score')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

## Preprocessing

In [ ]:
# Binary classification: Good (>=7) vs Not Good (<7)
df['label'] = (df['quality'] >= 7).astype(int)
df = df.drop(columns='quality')
X = df.drop(columns='label')
y = df['label']

## Simulating Semi-Supervised Learning

In [ ]:
# Mask 80% of labels
rng = np.random.RandomState(42)
y_masked = y.copy()
mask = rng.rand(len(y)) < 0.8
y_masked[mask] = -1
print(f"Labeled: {(y_masked != -1).sum()}, Unlabeled: {(y_masked == -1).sum()}")

## Self-Training Classifier

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train model
base_model = LogisticRegression(max_iter=1000)
self_training_model = SelfTrainingClassifier(base_model, threshold=0.8, verbose=True)
self_training_model.fit(X_scaled, y_masked)

## Evaluation

In [ ]:
y_pred = self_training_model.predict(X_scaled)
y_proba = self_training_model.predict_proba(X_scaled)[:, 1]

print(classification_report(y, y_pred))

# Confusion Matrix
sns.heatmap(confusion_matrix(y, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

# ROC Curve
roc_auc = roc_auc_score(y, y_proba)
print(f"ROC AUC Score: {roc_auc:.4f}")
RocCurveDisplay.from_predictions(y, y_proba)
plt.title("ROC Curve")
plt.tight_layout()
plt.show()